In [1]:
# Installing dependencies
import numpy as np
import pandas as pd
from yfQuery import datareader

In [2]:
class DataLoader:
    '''
        Class for loading past stocks prices
        symbol: symbol can be either a single symbol or 
                a list of symbol
        start:  start date
        end:    end date
    '''
    def __init__(self, symbol, start, end):
        # Preload
        self.data = datareader(symbol, start, end)
    
    def get(self, start, end):
        # Return a period of the data
        return self.data.iloc[start: end]

In [173]:
d = DataLoader('AAPL', '2016-01-01', '2019-12-31')
prices = d.data['Close']

In [174]:
# MA Crossing
a, b = 3, 5
ma14 = prices.rolling(a).mean()
ma30 = prices.rolling(b).mean()
data = pd.concat([prices, ma14, ma30], axis=1)
data.columns = ['Close', '14-days-ma', '30-days-ma']
data = data.dropna()
data['Above'] = data['14-days-ma'] > data['30-days-ma']
data

,Close,14-days-ma,30-days-ma,Above
Date,,,,
2016-01-08,24.240000,24.509166,25.108500,False
2016-01-11,24.632500,24.328333,24.767500,False
2016-01-12,24.990000,24.620833,24.630000,False
2016-01-13,24.347500,24.656667,24.464500,True
2016-01-14,24.879999,24.739166,24.618000,True
...,...,...,...,...
2019-12-24,71.067497,70.642499,70.373499,True
2019-12-26,72.477501,71.514999,70.881999,True
2019-12-27,72.449997,71.998332,71.370999,True


In [175]:
length = len(data)
holding = False
buy_at = 0
stat = []
dollar_gain = []
pct_gain = []

for i in range(length):
    is_above = data['Above'].iloc[i]
    price = data['Close'].iloc[i]
    if i + 1 == length:
        if holding:
            dollar_gain.append(price - buy_at)
            pct_gain.append(price / buy_at - 1)
            stat.append([np.nan, price])
            buy_at = 0
            holding = False
    else:
        if not holding:
            if is_above:
                buy_at = price
                stat.append([price, np.nan])
                holding = True
            else:
                stat.append([np.nan, np.nan])
        else:
            if not is_above:
                dollar_gain.append(price - buy_at)
                pct_gain.append(price / buy_at - 1)
                stat.append([np.nan, price])
                buy_at = 0
                holding = False
            else:
                stat.append([np.nan, np.nan])


In [176]:
stats = np.array(stat)
data['Buy'] = stats[:, 0]
data['Sell'] = stats[:, 1]
data

,Close,14-days-ma,30-days-ma,Above,Buy,Sell
Date,,,,,,
2016-01-08,24.240000,24.509166,25.108500,False,NaN,NaN
2016-01-11,24.632500,24.328333,24.767500,False,NaN,NaN
2016-01-12,24.990000,24.620833,24.630000,False,NaN,NaN
2016-01-13,24.347500,24.656667,24.464500,True,24.3475,NaN
2016-01-14,24.879999,24.739166,24.618000,True,NaN,NaN
...,...,...,...,...,...,...
2019-12-24,71.067497,70.642499,70.373499,True,NaN,NaN
2019-12-26,72.477501,71.514999,70.881999,True,NaN,NaN
2019-12-27,72.449997,71.998332,71.370999,True,NaN,NaN


In [177]:
num_trades = len(dollar_gain)
total_dollar_gain = sum(dollar_gain)
total_pct_gain = sum(pct_gain)
print('Gain ${:.2f} in {} trades per share'.format(total_dollar_gain, num_trades))
print('Capital percentage gain {:.2f}%'.format(total_pct_gain * 100))

Gain $21.10 in 107 trades per share
Capital percentage gain 53.55%
